In [1]:
from datasets import load_dataset, Dataset
from trl import SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from peft import LoraConfig
import torch
import pandas as pd
import json

treat the dataset, from raw log into conversational format (only need to run once)

In [2]:
llm_log = "../data/injected_log.csv"
normal_log = "../data/logs.csv"

In [3]:
def load_csv(dataset_path):
    df = pd.read_csv(dataset_path, skiprows=lambda x: x in range(1), names=['log', 'label', 'category', 'misc'])
    return df

In [4]:
df1 = load_csv(llm_log)
df2 = load_csv(normal_log)

In [5]:
df = pd.concat([df1, df2], ignore_index=True)
df = df.drop(df.columns[[2, 3]], axis=1)

In [6]:
def generate_prompt(log):
    messages = [
        {"role": "system", "content": "You are a cybersecurity expert analyzing Apache log entries to detect potential security threats."},
        {"role": "user", "content": "Given a log entry collected from Apache HTTP server, classify it as ""Malicious"" or ""Benign"". \n\
        If malicious, briefly explain why. If benign, just return the classification. Output must be in JSON format for structured parsing. \n\
        Format: \n\
        {{ \n\"classification\": \"Malicious or Benign\",\n\
        \"reason\": \"Explanation if malicious, otherwise empty\"\n\
        }}\n\
        Log:" + log},
    ]
    return messages
def generate_response(label):
    if label == 0:
        return {"role": "assistant", "content": "```json {{ \n \"classification\" : \"Benign\", \n \"reason\":\"\"\n}}\n```"}
    else:
        return {"role": "assistant", "content": "```json {{ \n \"classification\" : \"Malicious\", \n \"reason\":\"I think this is malicious!\"\n}}\n```"}

In [7]:
dicts = []
for _, row in df.iterrows():
    conversation = generate_prompt(row.iloc[0])
    conversation.append(generate_response(row.iloc[1]))
    entry = {"messages": conversation}
    dicts.append(entry)

with open("../data/prompt.json", "w", encoding="utf-8") as f:
    json.dump(dicts, f, indent=2, ensure_ascii=False)    

load dataset from json into huggingface Dataset format

In [6]:
dataset = load_dataset("json", data_files="../data/prompt.json", split='train')
dataset= dataset.train_test_split(test_size=0.2)

dataset

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 2486
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 622
    })
})

Training step: using LoRA with SFT trainer

In [3]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules="all-linear",
    modules_to_save=["lm_head", "embed_token"],
    task_type="CAUSAL_LM",
)

In [3]:
max_memory = {0: torch.cuda.get_device_properties(0).total_memory}
print(max_memory)

{0: 47725936640}


In [5]:
checkpoint='/rds/general/user/rm521/home/fyp/qwen2.5-7B'

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.bfloat16, max_memory=max_memory)
trainer = SFTTrainer(
    model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    args=SFTConfig(
        output_dir="Qwen2.5-7B-SFT", 
        do_eval=True,
        per_device_train_batch_size=2,
    ),
    peft_config=peft_config,
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.


Converting train dataset to ChatML:   0%|          | 0/2486 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2486 [00:00<?, ? examples/s]

...

Converting eval dataset to ChatML:   0%|          | 0/622 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/622 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/622 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/622 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [6]:
trainer.train()

Step,Training Loss
500,0.329900
1000,0.169400
1500,0.155700
2000,0.138300
2500,0.139300
3000,0.116600
3500,0.115900


TrainOutput(global_step=3729, training_loss=0.16331817062267448, metrics={'train_runtime': 1651.5234, 'train_samples_per_second': 4.516, 'train_steps_per_second': 2.258, 'total_flos': 8.762846369825587e+16, 'train_loss': 0.16331817062267448})

Evaluation step
if model is not loaded in VRAM, reload from file

In [9]:
torch.cuda.empty_cache()

In [4]:
model_name = '/rds/general/user/rm521/home/fyp/step3-sft/Qwen2.5-7B-SFT/checkpoint-3729'
sft_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16, max_memory=max_memory)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.


In [7]:
def collate_fn(batch):
    # Each `batch[i]` is a dictionary with a "messages" field
    refs = [sample["messages"][2] for sample in batch]
    prompts = [
        tokenizer.apply_chat_template(
            sample["messages"][:2],  # assuming system + user
            tokenize=False,
            add_generation_prompt=True
        )
        for sample in batch
    ]
    tokenized = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        padding_side='left',
        truncation=True,
    )
    tokenized["ref"] = refs
    return tokenized

# Step 2: Create DataLoader over a sliced dataset
subset = dataset["test"]
loader = DataLoader(subset, batch_size=5, collate_fn=collate_fn)

In [8]:
def bin_class(response):
    response = response.lower()
    for line in response.split("\n"):
        if "classification" in line:
            if "benign" in line:
                return 0
            if "malicious" in line:
                return 1
    return 2

In [11]:
eos_token_id = tokenizer.convert_tokens_to_ids("<|im_end|>")

total_cnt = 0
correct_cnt = 0

for batch in loader:
    refs = batch.pop("ref")
    batch = {k: v.to(sft_model.device) for k, v in batch.items()}
    
    with torch.no_grad():
        generated = sft_model.generate(
            **batch,
            max_new_tokens=512,
            eos_token_id=eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    # Slice off prompt inputs to get only the generated completion
    generated_ids = [
        output[len(input_ids):]
        for input_ids, output in zip(batch["input_ids"], generated)
    ]
    
    responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    # Print responses
    for gen, ref in zip(responses, refs):
        print("=== Response ===")
        print(gen)
        print("=== Ref ===")
        print(ref)
        total_cnt += 1
        if (bin_class(gen) == bin_class(str(ref))):
            correct_cnt += 1
        print('\n')
    print('\n')


=== Response ===
```json {{ 
 "classification" : "Malicious", 
 "reason":"I think this is malicious!"
}}
```
=== Ref ===
{'content': '```json {{ \n "classification" : "Malicious", \n "reason":"I think this is malicious!"\n}}\n```', 'role': 'assistant'}


=== Response ===
```json {{ 
 "classification" : "Malicious", 
 "reason":"I think this is malicious!"
}}
```
=== Ref ===
{'content': '```json {{ \n "classification" : "Malicious", \n "reason":"I think this is malicious!"\n}}\n```', 'role': 'assistant'}


=== Response ===
```json {{ 
 "classification" : "Malicious", 
 "reason":"I think this is malicious!"
}}
```
=== Ref ===
{'content': '```json {{ \n "classification" : "Malicious", \n "reason":"I think this is malicious!"\n}}\n```', 'role': 'assistant'}


=== Response ===
```json {{ 
 "classification" : "Malicious", 
 "reason":"I think this is malicious!"
}}
```
=== Ref ===
{'content': '```json {{ \n "classification" : "Malicious", \n "reason":"I think this is malicious!"\n}}\n```', 'ro

In [12]:
accuracy = correct_cnt / total_cnt
print(f"Accuracy: {accuracy}, Correct: {correct_cnt}, Total: {total_cnt}")

Accuracy: 0.9710610932475884, Correct: 604, Total: 622
